# Particle Filter Lab


## 1 - Packages
- numpy: is the fundamental package for scientific computing with Python.
- matplotlib: is a famous library to plot graphs in Python.
- cv2: computer vision lib
- glob: The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
- math: for mathematics functions

In [1]:
# importing cv2  and useful lib
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import math as m
import winsound
#cv2.startWindowThread() 

### 2- Read Images ###
    - Note: OpenCV, Images takes as not RGB but BGR
    - list.append: adds an item to the end of the list.
    - cv2.cvtColor(): convert BGR to RGB

In [2]:
class readimages:
    def __init__(self,no):
        folders = glob.glob(r'C:\Users\Admin\LabPethon\PF\img*')
        imagenames_list = []
        for folder in folders:
            for f in glob.glob(folder+'/*.png'):
                imagenames_list.append(f)

        read_images = [] 
        self.img_red = []
        self.img_rgb = []
        for image in imagenames_list:
            read_images.append(cv2.imread(image))
        self.img_rgb = [cv2.cvtColor(x, cv2.COLOR_BGR2RGB) for x in read_images]
        for i in range(no):
            img = self.img_rgb[0]
            height, width, layers = img.shape
            self.size = (width,height)
            self.img_red.append(self.img_rgb[i][:,:,0])

### 3- Test images
- cv2.imshow() to display an image in a window.
- cv2.waitKey() is a keyboard binding function. Its argument is the time in milliseconds. 

In [60]:
#plt.imshow(img_rgb[100])
#print('Shape of the image : {}'.format(img_red[100].shape))
# Plot image 1 ofthe red channel
cv2.imshow('Snake Tracking0',img_red[5]-img_red[4])
cv2.imshow('Snake Tracking1',img_red[5])
cv2.imshow('Snake Tracking2',img_red[4])
cv2.imshow('Snake Tracking3',img_red[10])
#cv2.imshow('Snake Tracking2',img_rgb[30]-img_rgb[29])
cv2.waitKey(0)

-1

## Funtions and Classes
* "_" : Get last result 
* "ctrl + /" : toggle comment 
* "\" : new line
* "numpy.argwhere": Find the indices of array elements that are non-zero, grouped by element. 
* " """ " : comment
* "astype" : To convert a float array to an integer array in python 

### randParticles 
Create random paritcles of size No

In [3]:
class randParticles:
    def __init__(self, No):
        # Create random particles
        self.p = []
        x = np.floor(200*np.random.random(No))
        y = np.floor(200*np.random.random(No))
        [self.p.append([x[i].astype(int),y[i].astype(int)]) for i in range(No)]

### move
Move particles randomly 

In [4]:
def move(xt):
    x = []
    random = np.floor(4*np.random.random(1))
    if random == 1: 
        x = np.add(xt, [1, 0])
    elif random == 2:
        x = np.add(xt, [0, 1])
    elif random == 3:
        x = np.add(xt, [-1, 0])
    else: 
        x = np.add(xt, [0, -1])
    return x.astype(int)

### weight
Give each particle weight depending on if it is on the snake or not

In [71]:
def weight(xt,zt):
    xt = np.abs(xt)
    if xt[0] == 0 or xt[0] > 199: 
        xt[0] = 0
    
    if xt[1] == 0 or xt[1] > 199:
        xt[1] = 0
        
    if zt[xt[0],xt[1]] > 0:
        w = 1
    else:
        w = 0;
    return w

### weight for the snake head

In [5]:
def weight(xt,dz, z_t1):
    
    xx,yy =np.where(dz == 255)
   # print(np.where(dz == 255))
    
    xt = np.abs(xt)
    if xt[0] == 0 or xt[0] > 199: 
        xt[0] = 0
    
    if xt[1] == 0 or xt[1] > 199:
        xt[1] = 0
        
    if xx == xt[0] and yy == xt[1]:
        w = 2
    elif z_t1[xt[0],xt[1]] == 255:
        w = 1
    else:
        w = 0;
    return w

### Plotting 
- plot image + particles

In [6]:
class plot:
    def __init__(self, p, img):
        for i in range(len(p)):
            # Window name in which image is displayed
            window_name = 'Image'

            # Center coordinates
            center_coordinates = (p[i][1],p[i][0])

            # Radius of circle
            radius = 1

            # Blue color in BGR
            color = (0,0,255)

            # Line thickness of 2 px , -1 for fill
            thickness = -1

            # Using cv2.circle() method
            # Draw a circle with blue line borders of thickness of 2 px
            self.image = cv2.circle(img, center_coordinates, radius, color, thickness)
                




In [376]:
new = plot([[100,5]],img_rgb[0])
cv2.imshow('image',img_rgb[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

## Algorithm

 ![title](pf.png)


## Basic Tracking Algorithm

In [127]:
class particle_filter:
    def __init__(self, x0, z_t):
        self.x_t = x0
        Ns = len(x0[:])                 # no of particles 
        red_part = []                   # red particles position
        w_t = np.ones((Ns,1))           # particles weight
        for m in range(Ns):
            # Update
            new = move(self.x_t[m])
            self.x_t[m] = new
            # Correction
            w_t[m] = weight(self.x_t[m],z_t)
            if w_t[m] == 1:
                red_part.append(self.x_t[m])
            #self.x_t[m] = np.multiply(w_t[m],self.x_t[m]).astype(int)
            #print(self.x_t[m] )
    # Resampling
        # Duplicate the red particles

        duplicate = 4
        if len(red_part) > 0:
            for m in range(len(red_part)):
#                 print('len red part = ' + repr(len(red_part)))
#                 print('size before add = ' + repr(len(self.x_t)))
                [self.x_t.append(red_part[m]) for i in range(duplicate)]
#                 print('size after add = ' + repr(len(self.x_t)))

            # Remove 50 of the red particles
            count = 1
            for i in range(len(w_t)):
                if count > duplicate*len(red_part):
                    break
                if w_t[i] == 0:
#                     print('size before sub = ' + repr(len(self.x_t)))
                    del self.x_t[i]
#                     print('size after sub = ' + repr(len(self.x_t)))
                    count += 1

        else:
            new = randParticles(Ns)
            self.x_t = new.p

## Head Tracking 

In [21]:
class particle_filter:
    def __init__(self, x0, dz, z_t1):
        self.x_t = x0
        Ns = len(x0[:])                 # no of particles 
        red_part = []                   # red particles position
        head = []
        w_t = np.ones((Ns,1))           # particles weight
        for m in range(Ns):
            # Update
            new = move(self.x_t[m])
            self.x_t[m] = new
            # Correction
            w_t[m] = weight(self.x_t[m],dz, z_t1)
            if w_t[m] == 2:
                head.append(self.x_t[m])
              #  print(self.x_t[m])
            elif w_t[m] == 1:
                #print("Snake")
                red_part.append(self.x_t[m])
    # Resampling
        # Duplicate the red particles

        duplicate = 3  # duplicate the normal particle
        duplicate_head = 4 # triple the head particle
        if len(head) > 0:
            #print("Head")
            #print(head)
           # print('size before head add = ' + repr(len(self.x_t)))
            for m in range(len(head)):
                [self.x_t.append(head[m]) for i in range(duplicate_head)]
           # print('size after head add = ' + repr(len(self.x_t)))
            # Remove no of the red particles
            count = 1
            for i in range(len(w_t)):
                if count > duplicate_head*len(head):
                    break
                if w_t[i] == 0:
                    del self.x_t[i]
                    count += 1
            #print('size after head sub = ' + repr(len(self.x_t)))
                    
        elif len(red_part) > 0:
            #print("Snake")
            #print('size before norm add = ' + repr(len(self.x_t)))
            for m in range(len(red_part)):
                [self.x_t.append(move(red_part[m])) for i in range(duplicate)]
            #print('size after norm add = ' + repr(len(self.x_t)))
            # Remove no of the red particles
            count = 1
            for i in range(len(w_t)):
                if count > duplicate*len(red_part):
                    break
                if w_t[i] == 0:
                    del self.x_t[i]
                    count += 1
            #print('size after norm sub = ' + repr(len(self.x_t)))

        else:
            new = randParticles(Ns)
            self.x_t = new.p

### Main Code

In [20]:
Ns = 200
No_img = 990
images_with_paritcles = []
print("Images Reading ... ")
imag = readimages(No_img+2)
print("Done ... Start Algorithm")
img_rgb = imag.img_rgb
img_red = imag.img_red
particles = randParticles(Ns)
dzs = [img_red[i+1]-img_red[i] for i in range(No_img + 1)]
for i in range(No_img):
    print(i)
    images = plot(particles.p,img_rgb[i])
    images_with_paritcles.append(images.image)
    pf =  particle_filter(particles.p,dzs[i],img_red[i+1])
    particles.p = pf.x_t
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
print("Thanks You for patience ")


Images Reading ... 
Done ... Start Algorithm
0
1
2
3
4
5
6
7
8
9
10
Snake
11
Snake
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
Snake
41
Snake
Snake
42
Snake
Snake
Snake
Snake
Snake
43
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
44
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
45
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
46
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
47
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
48
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Snake
Sn

## Make a Video

In [22]:
out = cv2.VideoWriter('Head Detection v2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, imag.size)
for i in range(len(images_with_paritcles)):
    out.write(images_with_paritcles[i])
cv2.destroyAllWindows()
out.release()


## Tests